In [38]:
import pinecone
import os
from dotenv import load_dotenv
from pinecone import Pinecone
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain import hub
from langchain_core.prompts import PromptTemplate
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_tool_calling_agent

In [17]:
# Load environment variables
load_dotenv()

True

# Setting up Tools

In [18]:
pc = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))
index = pc.Index("restaurant-info-medium-testing")

In [19]:
embeddings_model = OpenAIEmbeddings(api_key=os.getenv('OPENAI_API_KEY'),
                                        model="text-embedding-3-small")

In [20]:
vector_store = PineconeVectorStore(index=index, embedding=embeddings_model)

In [21]:
search = TavilySearchResults(max_results=3)

In [14]:
search.invoke("Where is sushi in Los Angeles")

[{'url': 'https://www.yelp.com/search?cflt=sushi&find_loc=Los+Angeles,+CA',
  'content': 'Best Sushi Bars in Los Angeles, CA - Sushi by Scratch Restaurants: LA, Iki Nori, Sushi Gen, Yeon Omakase and Sushi, Hato Sushi, Izakaya Osen - Los Angeles, Sushi Ye, Murakami, Kingyubu, Yunomi Handroll'},
 {'url': 'https://www.timeout.com/los-angeles/restaurants/best-sushi-restaurants-in-la',
  'content': "The best sushi in Los Angeles. Whether you're looking to splurge or save, the best L.A. sushi bars will satisfy your craving for immaculately cut raw fish. Tuesday August 27 2024."},
 {'url': 'https://www.yelp.com/biz/sushi-gen-los-angeles',
  'content': 'My girlfriend has been here a thousand times and she swears that it\'s the best sushi in Los Angeles for its price. It is her "worth it winner." Come here early before it opens because there will be a wait for this great deal. Read more. Angela P. Elite 24. Alhambra, CA. 77. 108. 292. Jul 11, 2024.'}]

In [30]:
retriever = vector_store.as_retriever(k=1)

In [32]:
retriever.invoke("where is sushi")[0]

Document(id='fc165d3d-c44c-470c-9217-b83cfe78a4cf', metadata={'address': '116 Japanese Village Plaza Mall, Los Angeles, CA 90012, USA', 'name': 'Sushi & Teri', 'place_id': 'ChIJizyr7UfGwoARt2VuQwQ0eIw', 'rating': 4.1}, page_content='Sushi & Teri is a small, inexpensive Japanese restaurant located in Little Tokyo, offering a variety of sushi rolls, ramen, miso soup, and cucumber salad. Customers generally feel that the food is decent but not exceptional, with notable mentions of the Alaska and Baja rolls, Washington roll, ramen served piping hot, and the cucumber salad. Key pros include friendly service, affordable prices, and the convenience of ordering via tablet, while cons revolve around long wait times, understaffing issues, and smaller portion sizes compared to nearby restaurants.')

In [33]:
retriever_tool = create_retriever_tool(
    retriever,
    "restaurant_search",
    "Search for information about restaurants in Los Angeles. If user prompts for restauraunts, USE THIS TOOL BY DEFAULT!",
)

In [35]:
tools = [search, retriever_tool]

In [36]:
llm = ChatOpenAI(model="gpt-4o-mini")

In [40]:
llm.invoke("where is lebron currently", temperature=0.1)

AIMessage(content="As of October 2023, LeBron James is playing for the Los Angeles Lakers in the NBA. If you're looking for the most current updates or specific details, please check the latest sports news sources.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 12, 'total_tokens': 53, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'stop', 'logprobs': None}, id='run-f07277dc-aefc-4422-9fc7-aafea256c381-0', usage_metadata={'input_tokens': 12, 'output_tokens': 41, 'total_tokens': 53})

## Creating Agent + Executor

In [41]:
print()